### SETUP INICIAL DO PROJETO

In [1]:

#importação das bibliotecase e pacotes necessários para a análise

import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pandas_gbq as gbq
import re
import seaborn as sns
from dotenv import load_dotenv
from google.cloud import bigquery
from google.cloud.bigquery_storage import BigQueryReadClient
from google.oauth2 import service_account


# carregar o .env com as credenciais
load_dotenv("/mnt/c/Users/wrpen/OneDrive/Desktop/df_lh/.env")


# detectar ambiente: como eu estou usando wsl-ubuntu, no VS Code  -  Windows, estava dando conflitos de path
if os.name == "nt":  # se Windows
    credentials_path = r"C:\Temp\desafiolh-445818-3cb0f62cb9ef.json"
else:  # se WSL/Linux
    credentials_path = "/mnt/c/Temp/desafiolh-445818-3cb0f62cb9ef.json"


# parâmetros injetados pelo Papermill ou definidos manualmente, caso não existam no ambiente
if 'tables_to_process' not in locals():
    tables_to_process = [
        "desafioadventureworks-446600.raw_data.production-location"       
    ]

if 'output_dataset' not in locals():
    output_dataset = "desafioadventureworks-446600.raw_data_cleaned"


# configs do cliente BigQuery: input de project e location de acordo com dados no Bigquery
credentials = service_account.Credentials.from_service_account_file(credentials_path)
client = bigquery.Client(credentials=credentials, project=os.getenv("BIGQUERY_PROJECT"), location="us-central1")


#doc: tables_to_process: lista de tabelas que serão processadas
#     output_dataset: nome do dataset onde os dados processados serão armazenados, neste caso, raw_data_cleaned

In [2]:
# Print com a tabela que vai ser processada nesse notebook

print("Tabelas a processar:", tables_to_process)

Tabelas a processar: ['desafioadventureworks-446600.raw_data.production-location']


In [3]:
# Nome do dataset no Bigquery com os dados brutos (.csv) extraídos pelo Meltano 
dataset_id = 'raw_data'
print(dataset_id)

# Lista de tabelas do dataset raw_data no Bigquery
tables = client.list_tables('raw_data')
print("Tabelas disponíveis:")
for table in tables:
    print(table.table_id)

raw_data
Tabelas disponíveis:
humanresources-employee
person-address
person-businessentity
person-person
person-stateprovince
production-location
production-product
production-productcategory
production-productcosthistory
production-productinventory
production-productsubcategory
purchasing-purchaseorderdetail
purchasing-purchaseorderheader
purchasing-vendor
sales-creditcard
sales-customer
sales-salesorderdetail
sales-salesorderheader
sales-salesperson
sales-salesterritory
sales-store


# Exploratory Data Analysis (EDA) e Data Cleaning

### Glossário dos dados:

O termo ''doc:'', situado no rodapé de algumas cells, indica algo como:

- documentação: documentar decisões, análises e resultados;

- abreviações de termos, como bkp, df, entre outros.

In [4]:
# Setup inicial do df para realizar a EDA 

pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 10000)
pd.options.display.float_format = '{:.2f}'.format


#doc: df = dataframe  

In [5]:
# Dicionário para armazenar os df processados
df_processados = {}

# Iteração das tabelas e armazenamento em df
for input_table in tables_to_process:
    print(f"Processando tabela: {input_table}")
    
    table_name = input_table.split(".")[-1].replace("-", "_")  
    
    print("Lendo os dados do BigQuery...")
    query = f"SELECT * FROM `{input_table}`"
    table_data = client.query(query).to_dataframe()
    
    df_processados[table_name] = table_data
    print(f"Tabela {table_name} processada e armazenada com sucesso.")


print("Todas as tabelas foram processadas com sucesso!")

Processando tabela: desafioadventureworks-446600.raw_data.production-location
Lendo os dados do BigQuery...
Tabela production_location processada e armazenada com sucesso.
Todas as tabelas foram processadas com sucesso!


In [6]:
# Listar todas as variáveis criadas dinamicamente
for table_name in df_processados.keys():
    print(f"Variável criada: {table_name}")  

Variável criada: production_location


In [7]:
# Atribuir o df a uma variável com nome mais simples
production_location = df_processados['production_location']

print(f"Colunas: {production_location.shape[1]}\nLinhas: {production_location.shape[0]}")

Colunas: 5
Linhas: 182


In [8]:
# Identificar duplicadas com base em 'locationid'
duplicadas = production_location[production_location.duplicated(subset=['locationid'], keep=False)]

# Verificar se existem duplicadas
if not duplicadas.empty:
    
    duplicadas_ordenadas = duplicadas.sort_values(by=['locationid', 'modifieddate'])

    print("duplicadas ordenadas:")
    print(duplicadas_ordenadas)
else:
    print("Não foram encontradas duplicadas.")


#doc*: ID da tabela

duplicadas ordenadas:
     locationid            name  costrate  availability              modifieddate
0             1       Tool Crib      0.00          0.00 2008-04-30 00:00:00+00:00
14            1       Tool Crib      0.00          0.00 2008-04-30 00:00:00+00:00
15            1       Tool Crib      0.00          0.00 2008-04-30 00:00:00+00:00
16            1       Tool Crib      0.00          0.00 2008-04-30 00:00:00+00:00
17            1       Tool Crib      0.00          0.00 2008-04-30 00:00:00+00:00
..          ...             ...       ...           ...                       ...
177          60  Final Assembly     12.25        120.00 2008-04-30 00:00:00+00:00
178          60  Final Assembly     12.25        120.00 2008-04-30 00:00:00+00:00
179          60  Final Assembly     12.25        120.00 2008-04-30 00:00:00+00:00
180          60  Final Assembly     12.25        120.00 2008-04-30 00:00:00+00:00
181          60  Final Assembly     12.25        120.00 2008-04-30 00:00:00+

In [9]:
# Remover duplicadas* 
production_location = production_location.drop_duplicates(subset=['locationid'], keep='last')

print(f"Linhas após remover duplicadas (baseando-se na última 'modifieddate'): {len(production_location)}")

#bkp dos dados brutos
raw_data_bkp_2_sem_duplicadas = production_location.copy()


#doc: bkp = backup (cópia)
#doc*: mantendo a última ocorrência com base em 'modifieddate', pois ela que indica a data da última modificação nos dados
#      Importante, pois se houver erro na ingestão (duplicação), mantém os dados íntegros.

Linhas após remover duplicadas (baseando-se na última 'modifieddate'): 14


In [10]:
# Ordenar e exibir o df por 'locationid'
production_location = production_location.sort_values(by=['locationid'])

print(production_location)

     locationid                    name  costrate  availability              modifieddate
25            1               Tool Crib      0.00          0.00 2008-04-30 00:00:00+00:00
37            2       Sheet Metal Racks      0.00          0.00 2008-04-30 00:00:00+00:00
49            3              Paint Shop      0.00          0.00 2008-04-30 00:00:00+00:00
61            4           Paint Storage      0.00          0.00 2008-04-30 00:00:00+00:00
73            5           Metal Storage      0.00          0.00 2008-04-30 00:00:00+00:00
85            6   Miscellaneous Storage      0.00          0.00 2008-04-30 00:00:00+00:00
97            7  Finished Goods Storage      0.00          0.00 2008-04-30 00:00:00+00:00
121          10           Frame Forming     22.50         96.00 2008-04-30 00:00:00+00:00
133          20           Frame Welding     25.00        108.00 2008-04-30 00:00:00+00:00
145          30        Debur and Polish     14.50        120.00 2008-04-30 00:00:00+00:00
157       

In [11]:
# Iterar por todas as colunas do df, para verificar valores ausentes

# Verificar valores ausentes na coluna
for column in production_location.columns:   
    missing_rows = production_location[production_location[column].isnull()]
    print(f"Coluna '{column}': {missing_rows.shape[0]} linhas ausentes.")
    
# Mostrar as primeiras linhas ausentes, se preciso for, limitar o head() para dar menos outputs ou limitar os outputs
    if not missing_rows.empty:
        print(f"Exibindo as primeiras linhas com valores ausentes em '{column}':")
        print(missing_rows.head(), "\n")
    else:
        print(f"Nenhuma linha com valores ausentes em '{column}'.\n")

Coluna 'locationid': 0 linhas ausentes.
Nenhuma linha com valores ausentes em 'locationid'.

Coluna 'name': 0 linhas ausentes.
Nenhuma linha com valores ausentes em 'name'.

Coluna 'costrate': 0 linhas ausentes.
Nenhuma linha com valores ausentes em 'costrate'.

Coluna 'availability': 0 linhas ausentes.
Nenhuma linha com valores ausentes em 'availability'.

Coluna 'modifieddate': 0 linhas ausentes.
Nenhuma linha com valores ausentes em 'modifieddate'.



In [12]:
# Valores únicos por coluna, para verificar se colunas como flags, normalmente booleanas, possuem apenas 1 ou 2 valores.

valores_unicos = production_location.nunique(dropna=False)

print("Valores únicos incluindo NaN:")
print(valores_unicos)

Valores únicos incluindo NaN:
locationid      14
name            14
costrate         7
availability     5
modifieddate     1
dtype: int64


In [13]:
#verificar informações do df
production_location.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14 entries, 25 to 181
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   locationid    14 non-null     Int64              
 1   name          14 non-null     object             
 2   costrate      14 non-null     float64            
 3   availability  14 non-null     float64            
 4   modifieddate  14 non-null     datetime64[us, UTC]
dtypes: Int64(1), datetime64[us, UTC](1), float64(2), object(1)
memory usage: 686.0+ bytes


In [14]:
# Atualizar o dicionário df_processados com o df ajustado
df_processados['production_location'] = production_location

In [15]:
# Padronizar colunas com valores textuais
production_location['name'] = production_location['name'].str.strip().str.upper()

print(production_location.head())

#doc: padronizar as strings nessa etapa, contribui para a execução das demais etapas do pipeline

    locationid               name  costrate  availability              modifieddate
25           1          TOOL CRIB      0.00          0.00 2008-04-30 00:00:00+00:00
37           2  SHEET METAL RACKS      0.00          0.00 2008-04-30 00:00:00+00:00
49           3         PAINT SHOP      0.00          0.00 2008-04-30 00:00:00+00:00
61           4      PAINT STORAGE      0.00          0.00 2008-04-30 00:00:00+00:00
73           5      METAL STORAGE      0.00          0.00 2008-04-30 00:00:00+00:00


In [16]:
# Garantir que apenas tabelas únicas sejam exportadas
unique_df_processados = {k: v for k, v in df_processados.items()}

# Exportar tabelas para o BigQuery
for table_name, df_cleaned in unique_df_processados.items():
 
    output_table = f"{output_dataset}.{table_name}"
   
    job_config = bigquery.LoadJobConfig(
        write_disposition="WRITE_TRUNCATE"  
    )
    
    job = client.load_table_from_dataframe(df_cleaned, output_table, job_config=job_config)
    job.result()

    print(f"Tabela {table_name} exportada com sucesso para {output_table}.")

Tabela production_location exportada com sucesso para desafioadventureworks-446600.raw_data_cleaned.production_location.
